In [3]:
%pylab
%matplotlib inline
from PIL import Image
import numpy as np

Using matplotlib backend: <object object at 0x000001DD70812DA0>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


Funktion, die die HSV Werte berechnet. Sie ist komisch und die HSV Werte würden mit denen aus einem Online-Converter nicht übereinstimmen. Nach viel Recherche habe ich herausgefunden, warum es so ist und warum die frühere Funktion (die eigentlich die Wikipedia Formel war) nicht für eine HSV-Farbraumkonversion in Python geeignet war.

Normalerweise sind bei HSV die Werte folgendermaßen verteilt: H zwischen 0 und 360, S und V zwischen 0 und 100 -> so werden die HSV Werte auch von Online-Convertern laut der theoretischen Formel berechnet

Diese Funktion ist aber anders. So werden die Werte verteilt:
H zwischen 0 und 1; 
S zwischen 0 und 1; 
V zwischen 0 und 255; 

zum Beispiel für RGB = (37, 16, 111) würde normalerweise die Formel (253, 85, 43) ergeben, aber die unten stehende Funktion ergibt (0.70, 0.85, 111) (die Werte haben viel mehrere Nachkommastellen, dem Übersicht halber habe ich nur 2 Nachkommastellen gelassen, aber für die folgenden Berechnungen würden alle Nachkommastellen Sinn ergeben)

Ein richtiges Output-Bild (nach dem Laufen des ganzen Code von unten) hat für diese RGB Werte folgende HSV Werte: (179, 218, 111).

Alles ganz verwirrend, oder? 

Es ist alles eine Sache der Skalierung:

Für H: 0.70... * 360 = 253 (Also die 0-360 Grade wurden auf 0-1 skaliert); 
Für S: 0.85... * 100 = 85 (Einfach 0-1 statt 0-100); 
Für V: 111 * 100 / 255 = 43 (*100 weil laut Formel ist V 0-100 und /255 weil laut Funktion V 0-255); 

Alles genau so wie die Formel, yay!

Dann warum hat das Output-Bild diese komischen HSV-Werte???

Anscheinend werden die HSV-Werte von allen Packages in Python, inklusive PIL oder matlab, nur in einem "255-Format" (so ähnlich wie RGB) unterstützt. Außerdem werden die H und S Werte in z.B. Matlab immer zwischen 0 und 1 angegeben. Deshalb verwendet man die untere Funktion, wo H und S zwischen 0 und 1 sind.
Um in "255-Format" zu konvertieren, muss man einfach um 255 multiplizieren.

H: 0.70... * 255 = 179; 
S: 0.85... * 255 = 218; 
V: ist sowieso zwischen 0 und 255; 


In [46]:
def rgb_to_hsv(r, g, b):
    
    maxc = max(r, g, b)
    minc = min(r, g, b)
    rangec = (maxc-minc)
    v = maxc
    if minc == maxc:
        return 0.0, 0.0, float(v)
    s = rangec / maxc
    rc = (maxc-r) / rangec
    gc = (maxc-g) / rangec
    bc = (maxc-b) / rangec
    if r == maxc:
        h = bc-gc
    elif g == maxc:
        h = 2.0+rc-bc
    else:
        h = 4.0+gc-rc
    h = (h/6.0) % 1.0

    return h, s, v

rgb_to_hsv(37, 16, 111)

(0.7035087719298246, 0.8558558558558559, 111)

Fancy Funktion für die RGB-to-HSV Konversion
Erklärung der Fancy Funktion:
- isistance: Bild muss als PIL Image eingelesen werden (open Funktion)
- RGB channels werden durch split Methode in 3 Banden (r, g und b) gesplittet
- leere Listen werden erstellt
- rd (von red), gn (green), bl (blue) 
- zip Methode erstellt Tupeln von den R, G und B Banden --> getdata extrahiert die Werte der jeweiligen Banden (R, G, B) --> so werden die RGB Werte für jedes Pixel in ein Tupel reingepackt
- Diese RGB Werte werden zu HSV konvertiert --> 
!!!
hier muss man durch 255 teilen (die theoretische Formel würde die RGB-Werte auf Werte zwischen 0 und 1 normalisieren, die obere Funktion nicht, deshalb teilen...) und dann wieder um 255 multiplizieren (um auf diesem 255 Format zu kommen)
!!!
- Die Werte der R, G und B Banden werden durch die HSV-Werte ersetzt. Achtung! Es bleiben trotzdem als r, g, b benannt, weil die 3 Banden am Anfang so definiert wurden
- Deshalb wird bei merge Funktion r, g, b eingegeben, aber Mode muss HSV sein

In [50]:
def HSVColor(img):
    if isinstance(img, Image.Image):
        r,g,b = img.split()
        Hdat = []
        Sdat = []
        Vdat = []
        for rd, gn, bl in zip(r.getdata(), g.getdata(), b.getdata()):
            h,s,v = rgb_to_hsv(rd/255., gn/255., bl/255.)
            Hdat.append(int(h*255.))
            Sdat.append(int(s*255.))
            Vdat.append(int(v*255.))
        r.putdata(Hdat)
        g.putdata(Sdat)
        b.putdata(Vdat)
        return Image.merge("HSV", (r,g,b))
    else:
        return None

HSV-Bilder können in Python nur angezeigt und !nicht! gespeichert werden! Deshalb muss man sie zuerst in ein Array umwandeln und dann das Array als .NPY speichern

In [58]:
RGB_image = Image.open("../data/Yeast_Cells.jpg")
HSV_image = HSVColor(RGB_image)
HSV_image.show() #anzeigen

In [59]:
#no need to run, Array ist schon gespeichert und gepusht
HSV_image_array_fancyfunction = np.asarray(HSV_image) #in array umwandeln
save("../data/HSV_YeastCells_array_fancyfunction", HSV_image_array_fancyfunction) #als array .npy speichern

Nicht so fancy Funktion (aber vielleicht verständlicher) mit array, das Output-Bild ist normal was die Farben angeht, aber an manchen Stellen verpixelter als die Fancy Funktion
Erklärung Funktion mit array:
- Bild wird in array umgewandelt
- Leere Liste wird erstellt
- für jedes Pixel werden die RGB Werte aus dem RGB_array in HSV Werte umgewandelt
- HSV Werte werden für jedes Pixel der Liste eingefügt (wieder die gleiche Geschichte mit 255 teilen und multiplizieren)
- Liste mit HSV Werten wird zu 3D-Array mit den gleichem Shape wie das Bild umgewandelt

In [54]:
def image_hsv_conversion_array(RGB_image):
    RGB_array = np.array(RGB_image)
    width, height = RGB_image.size
    HSV_list = []
    for x in range(width):
        for y in range(height):
            r, g, b = RGB_array[x,y]
            h, s, v = rgb_to_hsv(r/255., g/255., b/255.)
            listHSV = [int(h*255.), int(s*255.), int(v*255.)]
            HSV_list.append(listHSV)
    HSV_array = np.asarray(HSV_list, dtype=uint8).reshape(width, height, 3)
    return HSV_array

In [60]:
HSV_image_array_arrayfunction = image_hsv_conversion_array(RGB_image)
Image.fromarray(HSV_image_array_arrayfunction, "HSV").show() #anzeigen

In [61]:
save("../data/HSV_YeastCells_array_arrayfunction", HSV_image_array_arrayfunction) #als array .npy speichern

In [ ]:
#weil das Bild nicht gespeichert und als Array nicht auf K Means angewandt werden kann, muss ein Bild aus dem Array erstellt werden
array = np.load("../data/HSV_CellNuclei_array_arrayfunction.npy")
image = Image.fromarray(array, "HSV")